# Predictive Maintenance for Vehicle Fleets
*Note: If you are prompted to select a kernel, please select SageMaker Jumpstart PyTorch 1.0*

In this notebook you can get a quick preview of what the outcome when you complete the full notebook for this solution.

We have some sample time-series input data consisting of battery voltage and battery current over time. We have previously trained a model on this voltage and current data and have deployed the model as an endpoint in SageMaker. We will call this endpoint with some sample data to determine the probability of failure in the next time period.

You can select Run->Run All Cells from the menu to run all cells in Studio (or Cell->Run All in a SageMaker Notebook Instance).

In [ ]:
import boto3
import botocore
import json
import numpy as np
import matplotlib.pyplot as plt

from source.dl_utils.dataset import PMDataset_torch

with open("stack_outputs.json") as f:
    sagemaker_configs = json.load(f)

## Load and Visualize Sample Data

In [ ]:
test_ds = PMDataset_torch(
    "data/processed/demo.csv",
    sensor_headers=["voltage", "current"],
    target_column="target",
    standardize=True,
    verbose=False)

print("Sample input data:")

data, label = test_ds[0]
fig, axs = plt.subplots(1, len(["voltage", "current"]), figsize=(2000/108, 400/108), dpi=108)

for i, sensor_header in enumerate(["voltage", "current"]):
    axs[i].plot(data[:, i])
    axs[i].set_ylabel("{}".format(sensor_header))

## Make Predictions with Demo Endpoint

In [ ]:
config = botocore.config.Config(read_timeout=200)
runtime = boto3.client('runtime.sagemaker', config=config)

data = np.ones(shape=(1, 20, 2)).tolist()
payload = json.dumps(data)

response = runtime.invoke_endpoint(EndpointName=sagemaker_configs["SageMakerDemoEndpointName"],
                                   ContentType='application/json',
                                   Body=payload)
out = json.loads(response['Body'].read().decode())[0]

print("Given the sample input data, the predicted probability of failure is {:0.2f}%".format(100*(1.0-out[0]))) 

## Next Stage
We'll introduce the solution. [Click here to continue.](./1_introduction.ipynb).